**First, generate statistics on a multi-tab Excel workbook representing multiple sheets of survey data representing the ALS TDI CDC cohort.**

**What this does**

Loop through all fields and result in an Excel notebook with a tab for each tab represented in the data set.

Create a new dataframe based on the data dictionary.

If there is more than one row per Subject (Subject = unique identifier), first restructure from long to wide so that there is one row per subject.

Then create a separate descriptive analysis for each tab in the Excel file representing descriptive and categorical analytics for evry field in every survey.  

Save results in separate tabs as a "chartbook."

Repeat for Looker formatted files, which have a different format from the format of the original CDC cohort.

In [ ]:
import pandas as pd

# Define file paths
survey_data_path = 'XXXSURVEYDATAINPUT.xlsx'
output_path = 'XXXSURVEYDATAOUTPUT.xlsx'

# Load the survey data
survey_sheets = pd.read_excel(survey_data_path, sheet_name=None)

# Function to restructure from long to wide format
def restructure_long_to_wide(df):
    if 'Subject' in df.columns:
        # Pivot the data to wide format using 'Subject' as the index
        wide_df = df.pivot_table(index='Subject', aggfunc='first').reset_index()
        return wide_df
    return df

# Function to create descriptive analysis for categorical variables
def create_categorical_analysis(df):
    results = []
    for column in df.select_dtypes(include=['object', 'category']):
        counts = df[column].value_counts(dropna=False)
        percentages = (counts / len(df) * 100).round(2)
        total = counts.sum()
        analysis = pd.DataFrame({
            'Value': counts.index,
            'Count': counts.values,
            'Percentage': percentages.values,
            'Total': total
        })
        analysis['Variable'] = column
        results.append(analysis)
    if results:
        return pd.concat(results, ignore_index=True)
    return None

# Function to create descriptive analysis for numeric variables
def create_numeric_analysis(df):
    if not df.empty:
        description = df.describe(include='all').transpose()
        description['count'] = description['count'].astype(int)
        description['%'] = (description['count'] / len(df) * 100).round(2)
        return description
    return None

# Function to clean sheet names to meet Excel's constraints
def clean_sheet_name(name):
    # Remove invalid characters
    invalid_chars = ['\\', '/', '*', '[', ']', ':', '?']
    for char in invalid_chars:
        name = name.replace(char, '')
    # Truncate to 31 characters
    return name[:31]

# Initialize an Excel writer
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for sheet_name, df in survey_sheets.items():
        # Restructure from long to wide format
        wide_df = restructure_long_to_wide(df)

        # Perform numeric descriptive analysis
        numeric_description = create_numeric_analysis(wide_df)

        # Write the numeric descriptive analysis to Excel
        if numeric_description is not None:
            clean_name = clean_sheet_name(f'{sheet_name}_Numeric_Description')
            numeric_description.to_excel(writer, sheet_name=clean_name)

        # Perform categorical analysis
        categorical_analysis = create_categorical_analysis(wide_df)

        # Write the combined categorical analysis to Excel
        if categorical_analysis is not None:
            clean_name = clean_sheet_name(f'{sheet_name}_Categorical_Description')
            categorical_analysis.to_excel(writer, sheet_name=clean_name, index=False)

    # Ensure at least one sheet is visible
    if 'Sheet' in writer.book.sheetnames:
        std = writer.book['Sheet']
        writer.book.remove(std)
    if not writer.book.sheetnames:
        writer.book.create_sheet(title='Summary')

# Indicate that processing is complete
print("Descriptive analysis has been saved to the Excel workbook.")


Descriptive analysis has been saved to the Excel workbook.


**Read in updated data exported from Looker.**
First we convert the text file to CSV, although we could have done it straight from txt. 

In [ ]:
#This pivots all but conditions and family history. 
import pandas as pd

# Paths to your files
csv_file_path = 'XXXLOOKERDATAINPUT.csv'
output_file_path = 'XXXLOOKERDATAOUTPUT.xlsx'

# Load the CSV file
dblooker_data = pd.read_csv(csv_file_path)

# Create a new Excel writer object
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    for survey_title in dblooker_data['Survey Data Survey Title'].unique():
        # Filter data for the current survey title
        survey_data = dblooker_data[dblooker_data['Survey Data Survey Title'] == survey_title]

        # Pivot the data to align with the structure of the Excel sheets
        pivot_table = survey_data.pivot_table(
            index='Survey Data Participant ID',
            columns='Survey Data Question Title',
            values='Survey Data Answer Value',
            aggfunc='first'
        )

        # Write the pivot table to a sheet named after the survey title
        pivot_table.to_excel(writer, sheet_name=survey_title[:31])  # Excel sheet names are limited to 31 characters

print(f"Restructured data saved to: {output_file_path}")


In [ ]:
import pandas as pd

# Paths to your files
csv_file_path = 'XXXCONDITIONSINPUT.csv'
conditions_output_path = 'XXXCONDITIONSOUTPUT.xlsx'

# Load the CSV file
dblooker_data_updated = pd.read_csv(csv_file_path)

# Specify the condition titles we identified
condition_titles = ['Conditions 1', 'Conditions 2', 'Conditions 3', 'Conditions 4']

# Function to split conditions into separate columns
def separate_conditions(data, condition_column):
    # Split the conditions using the separator and expand into separate columns
    conditions_expanded = data[condition_column].str.split(' \| ', expand=True)

    # Reshape the data into a long format and remove NaN values
    conditions_melted = conditions_expanded.melt(value_name='Condition').dropna()

    # Get unique conditions
    unique_conditions = conditions_melted['Condition'].unique()

    # Create a new DataFrame with columns for each unique condition
    for condition in unique_conditions:
        if condition.startswith("Other - Write In"):
            # Extract and store the specific "Other" write-in response
            data[condition] = data[condition_column].apply(
                lambda x: x if condition in x else None)
        else:
            # Assign a boolean column indicating whether the condition was present
            data[condition] = data[condition_column].str.contains(condition, na=False)

    return data.drop(columns=[condition_column])

# Create a new Excel writer object
with pd.ExcelWriter(conditions_output_path, engine='xlsxwriter') as writer:
    for title in condition_titles:
        # Filter data for the current survey title
        survey_data = dblooker_data_updated[dblooker_data_updated['Survey Data Survey Title'] == title]

        if survey_data.empty:
            print(f"No data found matching the title '{title}'.")
            continue

        # Debugging: Check the first few entries to ensure data capture
        print(f"Data for {title}:")
        print(survey_data.head())

        # Restructure the data to focus on question titles
        try:
            # Create a DataFrame with Subject and Submitted columns
            subject_data = survey_data[['Survey Data Participant ID', 'Survey Data Date Submitted Date']].drop_duplicates()
            subject_data.columns = ['Subject', 'Submitted']

            # Pivot the data to create a wide format using the Question Title
            pivot_table = survey_data.pivot_table(
                index=['Survey Data Participant ID'],
                columns='Survey Data Question Title',
                values='Survey Data Answer Value',
                aggfunc=lambda x: ' | '.join(x.dropna().astype(str))  # Separate multiple entries with a different separator
            )

            # Debugging: Check pivot table structure
            print("Pivot Table Columns:")
            print(pivot_table.columns)
            print(pivot_table.head())

            # Reset index to merge
            pivot_table.reset_index(inplace=True)

            # Rename pivot table index for consistency
            pivot_table.rename(columns={'Survey Data Participant ID': 'Subject'}, inplace=True)

            # Merge the pivot table with the subject data
            merged_data = pd.merge(subject_data, pivot_table, on='Subject', how='left')

            # Debugging: Verify merged data
            print("Merged Data:")
            print(merged_data.head())

            # Get the last column name (conditions) to process it
            condition_col = merged_data.columns[-1]

            # Separate conditions into different columns, including "Other" responses
            processed_data = separate_conditions(merged_data, condition_col)

            # Write the processed data to a sheet named after the survey title
            processed_data.to_excel(writer, sheet_name=title[:31], index=False)  # Excel sheet names are limited to 31 characters
        except Exception as e:
            print(f"An error occurred while processing '{title}': {str(e)}")

print(f"Processed conditions data saved to: {conditions_output_path}")
